## Q1.

## 1-1.
首先通过蒙特卡罗方法来对期权进行定价，首先根据题干可知有10000条path，每条path有504个step，对于每条path，从1到504生成服从标准正态分布的随机变量$z_1, z_2, ..., z_{504}$

根据$S(t_{i+1}) = S(t_i) * exp\{(r - \frac{\sigma^2}{2})(t_{i+1}-t_i) + \sigma \sqrt{(t_{i+1}-t_i)}z_{i+1}\}$从$step_1$向$step_{504}$迭代计算

每条路径都可以得到一个股票均值，与行权价格比较得到这条路径的期权value，然后折现取均值，即是本题的定价结果

根据老师课件，$(1-\alpha)$置信区间可以通过如下公式计算，$\alpha$取0.05：

$\bar{c} = \frac{1}{n} \sum_{i=1}^{n}(c_i)$

$S^2 = \frac{1}{n-1} \sum_{i=1}^{n}(c_i - \bar{c})^2$

$(1-\alpha) \; confidence \; interval \; of \; c \; is \; (\bar{c}-t_{1-\frac{\alpha}{2},n-1} * \frac{S}{\sqrt{n}}, \; \bar{c}+t_{1-\frac{\alpha}{2},n-1} * \frac{S}{\sqrt{n}})$

通过scipy.stats.norm的interval()函数可以得到标准正态分布的置信区间，进而得到分位点，然后用上述公式来计算得到结果



## 1-2.
首先通过pathwise方法来计算$\Delta$和vega

计算$\Delta$的公式课件上已经给出，在蒙特卡罗模拟过程中可以同步计算出来每条path的结果，然后取均值作为估计结果：

$\alpha^{'}(S_0) = E[e^{-rT} \; \mathbb{1}\{\bar{S}>K\} \; \frac{\bar{S}}{S_0}]$  

<br/>

对于vega，推导如下：

$Y(S_0) = e^{-rT} \; \{\bar{S}-K\}^+$

$\frac{d}{d\sigma}Y(S_0)=e^{-rT}\mathbb{1}\{\bar{S}>K\} \frac{d}{d\sigma}\bar{S}$

$\frac{d}{d\sigma} \bar{S} = \frac{1}{m} \sum_{i=1}^{m}\frac{d}{d\sigma}S(t_i)$

$S(t_{i}) = S(0) * exp\{(r - \frac{\sigma^2}{2})t_i + \sigma \sqrt{(\Delta t)}(z_1 + ... + z_{i})\}$

$\frac{d}{d\sigma}S(t_i) = S(t_i)[-\sigma t_i + \sqrt{\Delta t} (z_1 + ... + z_{i})]$

代入得到

$\frac{d}{d\sigma}Y(S_0)=e^{-rT}\mathbb{1}\{\bar{S}>K\} \frac{1}{m} \sum_{i=1}^{m} S(t_i)[-\sigma t_i + \sqrt{\Delta t} (z_1 + ... + z_{i})]$

从而可以得到此时的$\alpha^{'}(S_0)$也即vega：

$\alpha^{'}(S_0) = E\{e^{-rT} * \mathbb{1}\{\bar{S}>K\} * \frac{1}{m} \sum_{i=1}^{m} S(t_i)[-\sigma t_i + \sqrt{\Delta t} (z_1 + ... + z_{i})]\}$

这样，在每条path的循环中的每个step，我们计算$S(t_i)[-\sigma t_i + \sqrt{\Delta t} (z_1 + ... + z_{i})]$即可

<br/>

对于$\Gamma$推导如下，记股票价格构成的向量$\overrightarrow{S}=(S_1,...,S_m)$，首先使用pathwise方法得到的$\Delta$结论

$\Gamma=\frac{\partial}{\partial S_0}\Delta=\frac{\partial}{\partial S_0}E[e^{-rT} * \mathbb{1}\{\bar{S}>K\} * \frac{\bar{S}}{S_0}]$

$=\frac{\partial}{\partial S_0}\int_{\overrightarrow{S}} e^{-rT} \mathbb{1}\{\bar{S}>K\} \frac{\bar{S}}{S_0} f(\overrightarrow{S};S_0)d\overrightarrow{S}$ 

$=E[e^{-rT} \mathbb{1}\{\bar{S}>K\} \bar{S} \frac{1}{f(\overrightarrow{S};S_0)} \frac{\partial}{\partial S_0}(\frac{f(\overrightarrow{S};S_0)}{S_0})]$

其中有

$\frac{\partial}{\partial S_0}(\frac{f(\overrightarrow{S};S_0)}{S_{0}})= \frac{1}{S_0^2}[\frac{\partial f(\overrightarrow{S};S_0)}{\partial S_0} S_0 - f(\overrightarrow{S};S_0)]$

代入回来得到

$\Gamma=E[e^{-rT} \mathbb{1}\{\bar{S}>K\} \bar{S} \frac{1}{f(\overrightarrow{S};S_0)} \frac{1}{S_0^2}[\frac{\partial f(\overrightarrow{S};S_0)}{\partial S_0} S_0 - f(\overrightarrow{S};S_0)]]$

$=E[e^{-rT} \mathbb{1}\{\bar{S}>K\} \frac{\bar{S}}{S_0^2} \frac{1}{f(\overrightarrow{S};S_0)} [\frac{\partial f(\overrightarrow{S};S_0)}{\partial S_0} S_0 - f(\overrightarrow{S};S_0)]]$

$=E[e^{-rT} \mathbb{1}\{\bar{S}>K\} \frac{\bar{S}}{S_0^2} [\frac{\partial log(f(\overrightarrow{S};S_0))}{\partial S_0} S_0 - 1]]$

根据课件，$\frac{\partial log(f(\overrightarrow{S};S_0))}{\partial S_0}=\frac{z_1}{S_0 \sigma \sqrt{\Delta t}}$

代入回来得到

$\Gamma=E[e^{-rT} \mathbb{1}\{\bar{S}>K\} \frac{\bar{S}}{S_0^2} (\frac{z_1}{ \sigma \sqrt{\Delta t}} - 1)]$

因此，在蒙特卡罗模拟的每条path中可以同步计算当前的$\Gamma$

In [7]:
import numpy as np
from scipy.stats import norm

In [40]:
num_replications = 10000

stock_price_0 = 40
strike_price = 40

total_months = 24
total_years = 2
total_trading_days = 504
delta_t = 1.0 * total_years / total_trading_days

volatility = 0.3
risk_free_rate = 0.025

In [41]:
def monte_carlo(num_replications, volatility, risk_free_rate, delta_t, total_years, total_trading_days, stock_price_0, strike_price):
    exponent_part1 = (risk_free_rate - 0.5 * volatility * volatility) * delta_t
    exponent_part2_without_z = volatility * np.sqrt(delta_t)
    multi_factor = np.exp(-1 * risk_free_rate * total_years)
    sqrt_delta_t = np.sqrt(delta_t)
    
    mid_stock_result = []
    pricing_result = []
    pathwise_vega_list = []
    pathwise_delta_list = []
    likelihood_gamma_list = []
    # likelihood_gamma_list1 = []

    for i in range(num_replications):
        mid_result_one_path = []
        stock_before = stock_price_0
        z_list = list(np.random.randn(total_trading_days))
        derivative_S_ti_with_sigma_list = []

        for j, cur_z in enumerate(z_list):
            stock_after = stock_before * np.exp(exponent_part1 + exponent_part2_without_z * cur_z)
            mid_result_one_path.append(stock_after)
            
            # 开始计算vega所需的中间变量
            vega_left_part = -1 * volatility * (j + 1) * delta_t # 因为j从0开始，所以求时间差要+1
            vega_right_part = sqrt_delta_t * np.sum(z_list[:(j + 1)])
            derivative_S_ti_with_sigma = stock_after * (vega_left_part + vega_right_part)
            derivative_S_ti_with_sigma_list.append(derivative_S_ti_with_sigma)
            
            stock_before = stock_after
    
        # 计算模拟出来的one path的股票价格平均值
        stock_price_mean = np.mean(mid_result_one_path)
        # 计算one path的期权价值
        value = 0.0
        indicator = 0.0  # 构造indicator
        if stock_price_mean > strike_price:  # 如果需要strike
            value = stock_price_mean - strike_price
            indicator = 1.0
        pricing_result.append(multi_factor * value) # 求出了one path的option price
        
        # 计算pathwise method下的vega和delta
        pathwise_one_path_vega = multi_factor * indicator * np.mean(derivative_S_ti_with_sigma_list)
        pathwise_one_path_delta = multi_factor * indicator * (stock_price_mean / stock_price_0)
        
        # 计算likelikhood ratio method下的gamma
        # 下面这个是直接用似然比的方法，凑二阶导数得到的结果
        # likelihood_factor_left1 = ((z_list[0] ** 2) - 1) / ((stock_price_0 ** 2) * (volatility ** 2) * delta_t)
        # likelihood_factor_right1 = z_list[0] / ((stock_price_0 ** 2) * volatility * sqrt_delta_t)
        # likelihood_gamma1 = multi_factor * value * (likelihood_factor_left1 - likelihood_factor_right1)
        # 下面这个是用pathwise得到的delta结论来做的
        likelihood_factor_left = (stock_price_mean * z_list[0]) / ((stock_price_0 ** 2) * volatility * sqrt_delta_t)
        likelihood_factor_right = stock_price_mean / (stock_price_0 ** 2)
        likelihood_gamma = multi_factor * indicator * (likelihood_factor_left - likelihood_factor_right)

        # 构造数据列表
        pathwise_vega_list.append(pathwise_one_path_vega)
        pathwise_delta_list.append(pathwise_one_path_delta)
        likelihood_gamma_list.append(likelihood_gamma)
        # likelihood_gamma_list1.append(likelihood_gamma1)
        # 添加one path的stock prices
        mid_stock_result.append(mid_result_one_path)

    return pricing_result, mid_stock_result, pathwise_delta_list, pathwise_vega_list, likelihood_gamma_list


In [42]:
print('进行5次蒙特卡罗模拟')
for cur_exp_index in range(0, 5, 1):
    pricing_result, mid_stock_result, pathwise_delta_list, pathwise_vega_list, likelihood_gamma_list = monte_carlo(
        num_replications, volatility, risk_free_rate, delta_t, total_years, 
            total_trading_days,stock_price_0, strike_price)
    
    option_price_mean = np.mean(pricing_result)

    option_price_array = np.array(pricing_result)
    var_option_price = np.sum((option_price_array - option_price_mean) 
        * (option_price_array - option_price_mean)) / (num_replications - 1)
    std_option_price = np.sqrt(var_option_price)

    t_left, t_right = norm.interval(0.95)
    interval_left = option_price_mean - t_right * std_option_price / np.sqrt(num_replications)
    interval_right = option_price_mean + t_right * std_option_price / np.sqrt(num_replications)
    interval_res = (interval_left, interval_right)

    pathwise_delta_res = np.mean(pathwise_delta_list)
    pathwise_vega_res = np.mean(pathwise_vega_list)
    likelihood_gamma_res = np.mean(likelihood_gamma_list)
    print('实验{} 期权价格为{:.4f} 95%置信区间为({:.4f}, {:.4f})'.format(
        cur_exp_index + 1,option_price_mean, interval_left, interval_right))
    print('其中delta:{:.4f} vega:{:.4f} gamma:{:.4f}'.format(pathwise_delta_res, 
        pathwise_vega_res, likelihood_gamma_res
        ))

进行5次蒙特卡罗模拟
实验1 期权价格为4.1908 95%置信区间为(4.0554, 4.3262)
其中delta:0.5557 vega:12.1027 gamma:0.0434
实验2 期权价格为4.2881 95%置信区间为(4.1473, 4.4288)
其中delta:0.5606 vega:12.5393 gamma:0.0462
实验3 期权价格为4.2294 95%置信区间为(4.0940, 4.3647)
其中delta:0.5676 vega:12.1617 gamma:0.0316
实验4 期权价格为4.2696 95%置信区间为(4.1323, 4.4069)
其中delta:0.5679 vega:12.3566 gamma:0.0618
实验5 期权价格为4.2231 95%置信区间为(4.0865, 4.3597)
其中delta:0.5615 vega:12.2127 gamma:0.0359


第一题解答：从上述模拟结果来看，期权价格大概在4.27美元左右，置信区间大概在(4.13, 4.41)左右，delta大约为0.57，vega大约为12.36，gamma大约为0.03-0.06左右的一个量，从实验结果来看gamma数值很小，结果不太稳定

## Q2

### 2-1.

(1)首先正向传播生成10000条路径，每条路径上有378个要生成的股票价格，记$b=10000, \; m=378$

使用的basis function为$\overrightarrow{\Psi}{(x)}=(1, \; x, \; x^2, \; x^3, \; x^4)$

(2)然后在terminal nodes计算最后的期权价值$\widetilde{V}_{m,j} = (K-S_{m,j})^+, \; j=1,...,b$

(3)然后使用least square regression求解系数$\beta$，对$(\overrightarrow{\Psi}(S_{m-1,j}), e^{-r\Delta t} * \widetilde{V}_{m,j})$进行回归

用系数$\beta$回来求$C_{m-1}(x)$，然后计算这一步的期权价值${\widetilde{V}}_{m-1,j}=max\{(K-S_{m-1,j})^+, C_{m-1}(S_{m-1,j})\}$，返回(2)迭代计算

(4)求到$j=1$时计算最终结果，${\widetilde{V}}_{0}=e^{-r\Delta t}* \frac{1}{b} \sum_{j=1}^{b}{\widetilde{V}}_{1,j}$

In [17]:
american_put_strike_price = 40.0
american_stock_price_0 = 40.0

american_total_months = 18.0
american_total_years = 18 / 12.0

american_volatility = 0.25
american_risk_free_rate = 0.025

american_replication_num = 10000
american_num_intervals = 378
american_delta_t = american_total_years / american_num_intervals

In [18]:
def american_option_monte_carlo(american_replication_num, american_num_intervals, 
    american_volatility, american_risk_free_rate, american_delta_t, 
    american_total_years, american_total_months, 
    american_stock_price_0, american_put_strike_price):
    # 为了与Q1的变量名区分开 注意变量名bug
    exponent_part1 = (american_risk_free_rate - 0.5 * american_volatility * american_volatility) * american_delta_t
    exponent_part2_without_z = american_volatility * np.sqrt(american_delta_t)
    # 注意在least square方法中是逐时刻discount，所以指数因子是delta_t
    multi_factor = np.exp(-1 * american_risk_free_rate * american_delta_t)
    sqrt_delta_t = np.sqrt(american_delta_t)
    
    mid_stock_result = []
    pricing_result = []
    z_list = []

    # 首先生成全部的股票价格
    stock_before = american_stock_price_0
    for i in range(american_num_intervals):
        cur_z_arr = np.random.randn(american_replication_num)  # TODO 存疑，变量独立吗
        z_list.append(cur_z_arr)  # 后续要transpose一下
        stock_after = stock_before * np.exp(exponent_part1 + exponent_part2_without_z * cur_z_arr)
        mid_stock_result.append(stock_after)
        stock_before = stock_after

    stock_arr = np.array(mid_stock_result).transpose()
    z_arr = np.array(z_list).transpose()
    # print('stock_arr shape: {shape}'.format(shape=stock_arr.shape))  # should be (10000, 378)
    # print('z_arr shape: {shape}'.format(shape=z_arr.shape))

    value_terminal = np.maximum(american_put_strike_price - stock_arr[:, -1], 0)
    # 此处仍然是一行 shape (10000, )
    value_after = value_terminal

    # 开始循环
    for j in range(american_num_intervals - 2, -1, -1):  # 378 days
        # stock_arr[:, 376]对应S_{m-1} stock_arr[:, 0]对应S_{1}
        cur_stock = stock_arr[:, j]  # shape (10000, )  
        discount_value = multi_factor * value_after  # 折合出当前时刻的option value
        
        # 开始构造psi
        psi_term_0 = (np.ones(american_replication_num)).reshape(1 , -1)
        psi_term_1 = (cur_stock).reshape(1, -1)
        psi_term_2 = (cur_stock ** 2).reshape(1 , -1)
        psi_term_3 = (cur_stock ** 3).reshape(1 , -1)
        psi_term_4 = (cur_stock ** 4).reshape(1 , -1)  # TODO 最高到4？
        cur_psi = np.concatenate((psi_term_0, psi_term_1, psi_term_2, psi_term_3, psi_term_4), axis=0).transpose()
        
        assert cur_psi.shape == (10000, 5)

        # 计算beta
        beta = np.linalg.lstsq(cur_psi, discount_value, rcond=None)[0]
        assert beta.shape == (5, )
        # 计算C_{m-1}(x)
        cur_C = np.dot(cur_psi, beta.reshape(-1, 1))[:, -1]  # (10000, 5) @ (5, 1)
        assert cur_C.shape == (10000, )

        # 开始求当前时刻的value
        strike_value = np.maximum(american_put_strike_price - cur_stock, 0)
        continuation_value = cur_C
        cur_value = np.maximum(strike_value, continuation_value)
        value_after = cur_value

    # 循环结束，计算结果
    value_0 = np.mean(value_after) * multi_factor

    return value_0


In [19]:
print('进行5轮实验')
for exp in range(5):
    res = american_option_monte_carlo(american_replication_num, american_num_intervals, 
        american_volatility, american_risk_free_rate, american_delta_t, 
        american_total_years, american_total_months, 
        american_stock_price_0, american_put_strike_price)
    print('本次蒙特卡罗模拟计算得到的期权价格为{res}'.format(res=res))

进行5轮实验
本次蒙特卡罗模拟计算得到的期权价格为5.08632737994784
本次蒙特卡罗模拟计算得到的期权价格为4.985458225118432
本次蒙特卡罗模拟计算得到的期权价格为5.019296255342759
本次蒙特卡罗模拟计算得到的期权价格为5.033956496559126
本次蒙特卡罗模拟计算得到的期权价格为4.97493998325035


第二题解答：从实验结果来看，得到的期权价格大概在5美元左右

## Q3

使用delta hedging策略，设卖出的期权对应$num\_shares$的股票

每个月进行rebalance，用当前时刻的delta与上个月的delta作差，0时刻可以认为上个月delta为0，作差结果的绝对值记为$delta\_change$，如果delta减小就卖出$delta\_change * num\_shares$的股票，如果delta增大则买入$delta\_change * num\_shares$的股票，即为当前时刻的cost，然后可以计算累积的cost，然后用累积的cost计算这个月的interest，这个月interest在下一次循环中也要计入到累积cost中去

在代码实现的时候，不需要算绝对值，因为作差结果带正负号，可以用相同的代码计算cost，负的cost代表这个月在卖出股票，print出来的中间过程的负的cost也是如此

In [8]:
num_shares = 100000.0
sold = 1200000.0
bsm_value = 997000.0

s_0 = 100.0
k = 100.0
r = 0.05
sigma = 0.35
T_months = 5.0
T_years = 5.0 / 12


In [9]:
stock_time_list = [100.0, 93.0, 87.0, 96.0, 105.0, 112.0]
call_time_list = [9.970, 5.318, 2.219, 4.116, 7.382, 12.000]
delta_time_list = [0.581, 0.430, 0.262, 0.438, 0.717, 1.000]

In [16]:
stock_time_mean = np.mean(stock_time_list)
stock_time_var = np.var(stock_time_list, ddof=1)

In [17]:
stock_time_mean

98.83333333333333

In [18]:
np.sqrt(stock_time_var)

8.886319072972041

In [3]:
rate_factor = 1.0 / 12 * r

In [4]:
delta_before = 0.0
cumulative_cost = 0.0
interest = 0.0
print('输出单位均为$1000')
for i in range(0, 6, 1):
    delta_change = delta_time_list[i] - delta_before
    stock_change = num_shares * delta_change
    cost_change = stock_time_list[i] * stock_change  # 本次rebalance买入或卖出cost多少
    cumulative_cost += (cost_change + interest)
    interest = cumulative_cost * (rate_factor)  # 用于下一次计算
    print('本次share purchased:{:.3f} cost:{:.3f} cumulative_cost:{:.3f} interest:{:.3f}'.format(
        stock_change / 1000, cost_change / 1000, cumulative_cost / 1000, interest  / 1000
    ))
    delta_before = delta_time_list[i]

输出单位均为$1000
本次share purchased:58.100 cost:5810.000 cumulative_cost:5810.000 interest:24.208
本次share purchased:-15.100 cost:-1404.300 cumulative_cost:4429.908 interest:18.458
本次share purchased:-16.800 cost:-1461.600 cumulative_cost:2986.766 interest:12.445
本次share purchased:17.600 cost:1689.600 cumulative_cost:4688.811 interest:19.537
本次share purchased:27.900 cost:2929.500 cumulative_cost:7637.848 interest:31.824
本次share purchased:28.300 cost:3169.600 cumulative_cost:10839.272 interest:45.164


In [5]:
final_cost = cumulative_cost - num_shares * k
target_profit = sold - bsm_value
final_profit = sold - final_cost
print('在t=5时刻 最终cost为{:.5f} 理想收益为{:.5f} 实际收益为{:.5f} 单位: $1000'.format(final_cost / 1000, 
    target_profit / 1000, 
    final_profit / 1000))

在t=5时刻 最终cost为839.27222 理想收益为203.00000 实际收益为360.72778 单位: $1000


第三题解答：最终的cost为839272美元，实际收益比理想收益要高，也就是计算结果中的最终的cost比BSM模型估算出来的价格要小，这可能和数据有关，上文中计算出了这几个月股价的标准差(代码为np.sqrt(stock_time_var))，发现达到了8.89，远高于题干的年波动率0.35。